# Removing Glasses from an image through recursive PCA

### Import Necessary Libraries

In [11]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

### Load Data

In [42]:
# Load Chosen Images as Dataset
def load_pgm_images(amount_of_people, pics_per_person, show_imgs=False):
    images = []
    for person in range(1, amount_of_people+1):
        images.append([])
        for image_number in range(1, pics_per_person+1):
            img = Image.open('images/s' + str(person) + '/' + str(image_number) + '.pgm')
            images[person-1].append(img)

    if show_imgs:
        rows, cols = len(images), len(images[0])
        fig, axes = plt.subplots(rows, cols)
        for i in range(rows):
            for j in range(cols):
                ax = axes[i, j]
                # img = Image.open(image_paths[i][j])
                ax.imshow(images[i][j], cmap='gray')
                ax.axis('off')
        plt.tight_layout()
        plt.show()

    return images

iamges = load_pgm_images(10, 10, False)

### Process Data

### Perform PCA on Glassless Dataset